# 2. Tokenization

Tokenization: split a document, any string, into discrete **tokens of meaning**.

本章主要介绍以下内容：

- 把一句话切成token / n-grams
- 处理特殊的标点，例如emoticons
- stemming / Lemmatization：进一步合并（压缩）所有token
- 对每句话创建一个vector representation
- sentiment analyzer from handcrafted token scores

## 1 Challenges

### 通常的预处理流程：

1. tokenization
    - separate punctuation from words
    - split contractions: we'll -> we will
    - emoticons
    - math symbols
    - **what is a token?**
        - ice cream: 1 token or 2 tokens? 
    
2. stemming
    - 根据token 的syllables, prefix, suffix
    - processing, processed, process
    - 困难：
        - remove ing
            - ending -> end
            - running -> run (not runn)
            - sing -> s (wrong)
        - plural form:
            - words -> word
            - bus -> bu (wrong)
        - more info: ch05_word2vec

3. invisible words
    - don't (do that)!
    
4. n-grams
    - including pairs of words
    - filter out n-grams that rarely occur together (low frequency)
    - 留下一些常用的组合，例如 ice cream, Mr. Smith


## 2. Building your vocabulary with a tokenizer

#### natural language processing v.s. programming language compiler

A tokenizer used for compiling computer languages is often called a **scanner** , **lexical analyzer** or **lexer**.
The vocabulary (the set of all the valid tokens) for a computer language is often called a **lexicon**

| Natural Language Processing   | Parser   | Tokenizer                        | Vocabulary |
|-------------------------------|----------|----------------------------------|------------|
| Programming Language Compiler | Compiler | Scanner, Lexer, Lexical Analyzer | Lexicon    |

A tokenizer breaks unstructured data, natural language text, into chunks of information that can be counted as discrete elements. (unstructured string -> numerical data structure)

#### 最简单的tokenizer: 使用空格

In [1]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."

In [7]:
print(sentence.split())

['Thomas', 'Jefferson', 'began', 'building', 'Monticello', 'at', 'the', 'age', 'of', '26.']


In [8]:
print(str.split(sentence))

['Thomas', 'Jefferson', 'began', 'building', 'Monticello', 'at', 'the', 'age', 'of', '26.']


从上面的例子可以看出，最后一个26后面的句号没有被切分开。通常，word 和punctuation 要分开。后面我们会逐步优化tokenizer。下面，我们先focus on pipeline.

### one-hot vector


In [4]:
import numpy as np
token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))

In [6]:
print(vocab)

['26.', 'Jefferson', 'Monticello', 'Thomas', 'age', 'at', 'began', 'building', 'of', 'the']


In [15]:
num_tokens = len(token_sequence)
print('the corpus has {} tokens.'.format(num_tokens))

vocab_size = len(vocab)
print('the corpus has {} unique tokens (vocab size).'.format(vocab_size))

onehot_vectors = np.zeros((num_tokens, vocab_size), int)  # #rows = #token, #cols = |vocab|
for i, word in enumerate(token_sequence):
    onehot_vectors[i, vocab.index(word)] = 1
    print('The {}th word: {}\tone-hot: {}'.format(i, word, onehot_vectors[i]))

the corpus has 10 tokens.
the corpus has 10 unique tokens (vocab size).
The 0th word: Thomas	one-hot: [0 0 0 1 0 0 0 0 0 0]
The 1th word: Jefferson	one-hot: [0 1 0 0 0 0 0 0 0 0]
The 2th word: began	one-hot: [0 0 0 0 0 0 1 0 0 0]
The 3th word: building	one-hot: [0 0 0 0 0 0 0 1 0 0]
The 4th word: Monticello	one-hot: [0 0 1 0 0 0 0 0 0 0]
The 5th word: at	one-hot: [0 0 0 0 0 1 0 0 0 0]
The 6th word: the	one-hot: [0 0 0 0 0 0 0 0 0 1]
The 7th word: age	one-hot: [0 0 0 0 1 0 0 0 0 0]
The 8th word: of	one-hot: [0 0 0 0 0 0 0 0 1 0]
The 9th word: 26.	one-hot: [1 0 0 0 0 0 0 0 0 0]


为了更好的可视化，我们使用**Pandas**库. 
使用pandas，每一行代表一个token 的one-hot vector.

In [14]:
import pandas as pd
pd.DataFrame(onehot_vectors, columns=vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


One-hot 可以想象成在弹钢琴，每一个键代表一个token，依次排开。每次只能弹一个键。例如，第一次弹了第4个键(Thomas), 第二次弹第2个键(Jefferson), 以此类推。


通过one-hot vector，我们把一个token 转换成了numbers，以便计算机理解和计算。
这种One-hot vector 通常用于：

- used in neural nets
- sequence-to-sequence language models
- generative language models
- etc.

#### **Point++**: 没有information lost 
除了空格无法恢复，但是空格本身带的信息很少).

#### **Point--**: one-hot vector representation 得到一个十分稀疏的矩阵。

对于实际应用场景，impractical. 下面我们预估以下：
- 假设我们有3000 books, 每本书有3500行，每行15个单词，所以我们表格的总行数为：

`#rows = 3000*3500*15 = 157500000`

假设我们vocab 有 100,000 个单词，那我们表格的总列数为：

`#cols = 100000`

假设matrix 的每个cell 用1个byte 来存储，则matrix 的总大小为：

`#size_in_byte = 157500000 * 100000 = 15750000000000`

所以总大小是 15.75 TB

`size_in_byte / le12 = 15.75`

所以我们要对这样one-hot matrix 做**dimension reduction**.



### Bag-of-Word

许多时候我们发现，词的顺序其实并不太影响我们理解一句话的意思。

所以我们又一个假设：the meaning of a sentence can be gleaned from just the words themselves. 

所以我们可以把所有的单词放在一个bag中，忽略单词之间的顺序。每一个bag 代表一个document，例如，一句话。

bag of word vector 可以通过把一个document 中每个token 的one-hot vector 相加得到（OR operation）。所以bag of word vector count the **frequency** of words, **not order**. 以前是每个token 一个长度为|vocab| 的向量，现在是每个document 一个长度为|vocab| 的向量。

one-hot vector 好比solo，一次只弹一个键；bag of word 更像和旋，一次可以同时弹多个键。

所以通过使用bag of word，我们压缩了初步的one-hot matrix.

除此以外，可以使用bag of word 来index document，因为使用bag of word vector 我们可以快速知道某一个document 中是否包含某一个单词。

#### set of word

下面的代码实现了set of word (区别在于，每个token 只有0:没有出现和1:出现两种状态，而不计数)

对于set of word，每个vector 是一个binary vector (0 and 1). Binary vector 的优势是，All modern CPUs have hard
wired memory addressing instructions that can efficiently hash, index, and search a large set of binary vectors like this.

In [47]:
sentence_bow = {}
for token in sentence.split():
    sentence_bow[token] = 1

print(sorted(sentence_bow.items()))

[('26.', 1), ('Jefferson', 1), ('Monticello', 1), ('Thomas', 1), ('age', 1), ('at', 1), ('began', 1), ('building', 1), ('of', 1), ('the', 1)]


因为每个token 的计数只可能是0或者1，所以我们可以直接使用一个set 来存储所有出现过的token，这样更省空间。

In [17]:
import pandas as pd
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T

In [18]:
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


下面，我们在corpus 中加一些句子。

In [20]:
sentences = "Thomas Jefferson began building Monticello at the age of 26.\n"
sentences += "Construction was done mostly by local masons and carpenters.\n"
sentences += "He moved into the South Pavilion in 1770.\n"
sentences += "Turning Monticello into a neoclassical masterpiece was Jefferson's obsession."

print(sentences)

Thomas Jefferson began building Monticello at the age of 26.
Construction was done mostly by local masons and carpenters.
He moved into the South Pavilion in 1770.
Turning Monticello into a neoclassical masterpiece was Jefferson's obsession.


In [26]:
import pprint

corpus = {}

"""
Normally you should use .splitlines() 
but here you explicitly add a single '\n' character to the end of each line/
sentence, so you need to explicitly split on this character.
"""
for i, sent in enumerate(sentences.split('\n')):  # 对每一句话生成一个beg of word vector
    corpus['sent{}'.format(i)] = dict((tok, 1) for tok in sent.split())
    
pp = pprint.PrettyPrinter()
pp.pprint(corpus)

{'sent0': {'26.': 1,
           'Jefferson': 1,
           'Monticello': 1,
           'Thomas': 1,
           'age': 1,
           'at': 1,
           'began': 1,
           'building': 1,
           'of': 1,
           'the': 1},
 'sent1': {'Construction': 1,
           'and': 1,
           'by': 1,
           'carpenters.': 1,
           'done': 1,
           'local': 1,
           'masons': 1,
           'mostly': 1,
           'was': 1},
 'sent2': {'1770.': 1,
           'He': 1,
           'Pavilion': 1,
           'South': 1,
           'in': 1,
           'into': 1,
           'moved': 1,
           'the': 1},
 'sent3': {"Jefferson's": 1,
           'Monticello': 1,
           'Turning': 1,
           'a': 1,
           'into': 1,
           'masterpiece': 1,
           'neoclassical': 1,
           'obsession.': 1,
           'was': 1}}


In [27]:
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.,...,South,Pavilion,in,1770.,Turning,a,neoclassical,masterpiece,Jefferson's,obsession.
sent0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0,...,1,1,1,1,0,0,0,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


### Similarity between sentence

One way to check for the similarities between sentences is to count the number of overlapping tokens using a **dot product**. (also called the scalar product because it produces a single scalar value as its output)

In [28]:
v1 = pd.np.array([1, 2, 3])
v2 = pd.np.array([2, 3, 4])
v1.dot(v2)

20

In [29]:
(v1 * v2).sum()  # fast

20

In [30]:
sum([x1 * x2 for x1, x2 in zip(v1, v2)])  # slow 

20

In [32]:
# use numpy matrix product operatior, np.matmul() function or the @ operator
v1.reshape(-1, 1).T @v2.reshape(-1, 1)  

array([[20]])

In [33]:
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.,...,South,Pavilion,in,1770.,Turning,a,neoclassical,masterpiece,Jefferson's,obsession.
sent0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0,...,1,1,1,1,0,0,0,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1


In [38]:
df = df.T
df

,sent0,sent1,sent2,sent3
Thomas,1,0,0,0
Jefferson,1,0,0,0
began,1,0,0,0
building,1,0,0,0
Monticello,1,0,0,1
at,1,0,0,0
the,1,0,1,0
age,1,0,0,0
of,1,0,0,0
26.,1,0,0,0


In [39]:
df.sent0.dot(df.sent1)

0

In [40]:
df.sent0.dot(df.sent2)

1

In [41]:
df.sent0.dot(df.sent3)

1

In [42]:
df.sent1.dot(df.sent2)

0

In [43]:
df.sent1.dot(df.sent3)

1

In [44]:
df.sent3.dot(df.sent2)

1

下面，我们可以查看同时出现的词。

In [46]:
[(k, v) for (k, v) in (df.sent2 & df.sent3).items() if v]

[('into', 1)]

### A token improvement

很多时候，我们不仅仅需要根据空格切分，需要根据其他的特殊字符切分。

In [48]:
import re

tokens = re.split(r'[-\s.,;!?]+', sentence)
print(tokens)

['Thomas', 'Jefferson', 'began', 'building', 'Monticello', 'at', 'the', 'age', 'of', '26', '']


### regex

注意，当我们需要通过-字符切分时，-字符必须放在open bracket 右侧，即第一个。原因是-在一个character class 中有特殊的意义，例如:r'[A-Z]' 表明匹配任意一个大写字符。

`re.split` 和`str.split`的行为类似，只是根据regex 的正则表达式的匹配作为分隔符。

- `[...]` 定义一个character class，匹配任意一个即可
- `(...)` 定义一个regex group，需要全部匹配

#### compiled regex 的好处：
- 更快：Python caches the compiled objects for the last MAXCACHE=100 regular expressions.

然后，我们尝试移除无效的token，例如空字符等。

In [49]:
pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)
[x for x in tokens if x and x not in '- \t\n.,;!?']

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

In [52]:
# 第二种方法，使用filter
list(filter(lambda x: x if x and x not in '- \t\n.,;!?' else None, tokens))

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

### tokenizer libraries:

- spaCy — Accurate , flexible, fast, Python
- Stanford CoreNLP — More accurate, less flexible, fast, depends on Java 8
- NLTK — Standard used by many NLP contests and comparisons, popular, Python

我们下面首先来看一个NLTK tokenizer

In [54]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+|$[0-9.]+|\S+')
tokenizer.tokenize(sentence)

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '.']

It also separates sentence-ending trailing punctuation from tokens that do not contain any other punctuation characters.

#### TreeBank Tokenizer

- separates phrase-terminating punctuation
- English contractions: isn't = is n't
    - 用途：syntax tree

In [56]:
from nltk.tokenize import TreebankWordTokenizer
new_sentence = "Monticello wasn't designated as UNESCO World Heritage Site until 1987."
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(new_sentence)

['Monticello',
 'was',
 "n't",
 'designated',
 'as',
 'UNESCO',
 'World',
 'Heritage',
 'Site',
 'until',
 '1987',
 '.']

#### Casual Tokenizer



In [57]:
from nltk.tokenize.casual import casual_tokenize
message = "RT @TJMonticello Best day everrrrrrr at Monticello. Awesommmmmmeeeeeeee day :*)"""
casual_tokenize(message)

['RT',
 '@TJMonticello',
 'Best',
 'day',
 'everrrrrrr',
 'at',
 'Monticello',
 '.',
 'Awesommmmmmeeeeeeee',
 'day',
 ':*)']

In [58]:
# reduce_len: reduce the number of repeated characters within a token
# strip_handles: strip usernames
casual_tokenize(message, reduce_len=True, strip_handles=True)

['RT',
 'Best',
 'day',
 'everrr',
 'at',
 'Monticello',
 '.',
 'Awesommmeee',
 'day',
 ':*)']